# Is dotnet 8 faster than dotnet 7?

`Expressur.LoadTest` is a simple executable that runs a very simple load test for Expressur. It does as many iterations as it can in five minutes of the following three expressions:

~~~csharp
        var expressions = new Dictionary<string, string> {
            { "cplusaplusb", "c + aplusb" },
            { "aplusb", "a + b" },
            { "extraindirection", "(aplusb/ cplusaplusb)" }
        };
~~~

It uses these values as initial seed values

~~~csharp
        var context = new Dictionary<string, decimal?> {
            { "a", 1m },
            { "b", 2m },
            { "c", 4m }
        };
~~~

"As many iterations as it can" is somewhat qualified. To both simplify the code and prevent the load test code from adding too much overhead that would skew the results, it runs in batches of 1 million. After every million batches, it checks if the total run time is greater than 5 minutes. If so, it ends. i.e.:

~~~csharp
    while (time < 5 minutes>){
        run 1 million batches
    }   
~~~

As a result, the run time will always be slightly greater than 5 minutes except in that rare case when the last batch was completed at exactly 5 minutes.

Running the same code built for dotnet 7 and 8, we saw these results:

In [17]:
from datetime import timedelta
from pandas import DataFrame

net7 = {"iterations":69_000_000, "total_time": timedelta(minutes=5, seconds=4.7028801)}
net8 = {"iterations":79_000_000, "total_time": timedelta(minutes=5, seconds=00.2671345)}

testresults = {"dotnet7": net7, "dotnet8": net8}


net7["time_per_million_iterations"] = net7["total_time"] / (net7["iterations"] / 1_000_000)
net8["time_per_million_iterations"] = net8["total_time"] / (net8["iterations"] / 1_000_000)

# display testresults as table
print("Test results:")

df = DataFrame(testresults)

improvement = 1 - (net8['time_per_million_iterations']/ net7['time_per_million_iterations'])

display(df)

# round improvement to 2 decimal places 
improvement = round(improvement * 100, 2)

print(f"Improvement: {improvement}%")


Test results:


,dotnet7,dotnet8
iterations,69000000,79000000
total_time,0:05:04.702880,0:05:00.267134
time_per_million_iterations,0:00:04.415984,0:00:03.800850


Improvement: 13.93%


An almost **14% improvement** is impressive when no code in Expressur changed at all.